# 01 — Model Verification: 5-Story RC Frame (OpenSeesPy)

**Project:** Hybrid Digital Twin for Seismic RC Buildings  
**Purpose:** Validate the nonlinear finite element model by inspecting the NLTHA output  
from the synthetic ground motion test (`Synthetic_Test.csv`).

**What we verify:**
1. The input ground motion is physically reasonable (PGA, duration, frequency content).
2. Inter-story drift ratios (IDR) are within expected damage levels.
3. The global hysteresis loop shows nonlinear behavior (stiffness degradation, energy dissipation).

All figures are exported at **300 DPI** via `FigureManager` for direct inclusion in the  
HRPUB manuscript (Section 4 — Results).

## 1. Data Loading & Editorial Style Configuration

In [ ]:
from __future__ import annotations

import json
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Ensure project root is on sys.path so we can import src.*
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.figure_manager import FigureManager  # noqa: E402

# ── Style: HRPUB-compliant (serif, 300 DPI, seaborn grid) ──────────────
plt.style.use("seaborn-v0_8-whitegrid")

fig_mgr = FigureManager(
    output_dir=str(PROJECT_ROOT / "figures"),
    dpi=300,
    fmt="png",
    figsize=(7.0, 4.5),  # double-column width
)

# ── Load NLTHA results ─────────────────────────────────────────────────
DATA_DIR = PROJECT_ROOT / "data" / "raw"
csv_path = DATA_DIR / "Synthetic_Test.csv"
meta_path = DATA_DIR / "Synthetic_Test_meta.json"

df = pd.read_csv(csv_path)

with open(meta_path, encoding="utf-8") as f:
    meta = json.load(f)

print(f"Dataset loaded: {len(df)} time steps, {len(df.columns)} columns")
print(f"Time range: {df['time'].iloc[0]:.3f} – {df['time'].iloc[-1]:.3f} s")
print(f"Columns: {list(df.columns)}")
print(f"\nGround motion: {meta['ground_motion']['name']}")
print(f"Duration: {meta['ground_motion']['duration']:.2f} s, dt = {meta['ground_motion']['dt']} s")
print(f"Converged: {meta['results']['converged']}")
print(f"Wall-clock: {meta['results']['wall_clock_s']:.1f} s")

## 2. Seismic Input: Synthetic Ground Motion

The input acceleration time history contextualizes the structural demand.  
We annotate the **Peak Ground Acceleration (PGA)** — the single most important  
intensity measure for code-based assessment (ASCE 7-22 §11.4).

In [ ]:
# ── Input ground motion ────────────────────────────────────────────────
time = df["time"].values
acc = df["ground_accel"].values

pga = float(np.max(np.abs(acc)))
pga_time = time[int(np.argmax(np.abs(acc)))]

fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(time, acc, color="black", lw=0.8)

# Annotate PGA
ax.annotate(
    f"PGA = {pga:.2f} m/s² ({pga / 9.81:.3f}g)",
    xy=(pga_time, acc[int(np.argmax(np.abs(acc)))]),
    xytext=(pga_time + 1.5, pga * 0.85),
    fontsize=9,
    arrowprops=dict(arrowstyle="->", color="red", lw=1.2),
    color="red",
    fontweight="bold",
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="red", alpha=0.9),
)

ax.set_title("Seismic Input: Synthetic Ground Motion")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Acceleration (m/s²)")
ax.set_xlim(0, time[-1])

fig_mgr.save(fig, caption=f"Synthetic input ground motion time history (PGA = {pga / 9.81:.3f}g).")
plt.show()
print(f"PGA = {pga:.3f} m/s² = {pga / 9.81:.3f}g at t = {pga_time:.2f} s")

## 3. Inter-story Drift Ratio (IDR) History

The IDR is the primary Engineering Demand Parameter (EDP) for seismic assessment.  
Drift limits per ASCE 41-17:
- **< 0.5%** — Immediate Occupancy (IO)
- **< 1.0%** — Life Safety (LS)
- **< 2.0%** — Collapse Prevention (CP)

Our IDR_max ≈ 1.24% places the structure in the **Life Safety–CP transition zone**,  
confirming controlled nonlinear behavior — ideal training data for the PINN.

In [ ]:
# ── Inter-story drift ratio time histories ─────────────────────────────
n_stories = 5
story_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd"]

fig, ax = plt.subplots(figsize=(10, 5))

idr_max_values: dict[int, float] = {}
for i in range(1, n_stories + 1):
    drift_pct = df[f"drift_{i}"].values * 100  # → %
    ax.plot(time, drift_pct, color=story_colors[i - 1], lw=1.0, label=f"Story {i}")
    idr_max_values[i] = float(np.max(np.abs(drift_pct)))

# Performance limits
ax.axhline(y=1.0, color="red", ls="--", lw=1.0, label="Life Safety (1.0%)")
ax.axhline(y=-1.0, color="red", ls="--", lw=1.0)
ax.axhline(y=2.0, color="darkred", ls=":", lw=0.8, label="Collapse Prevention (2.0%)")
ax.axhline(y=-2.0, color="darkred", ls=":", lw=0.8)

ax.set_title("Inter-story Drift Ratio (IDR) History")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Drift Ratio (%)")
ax.legend(loc="upper right", fontsize=8, ncol=2)
ax.set_xlim(0, time[-1])

fig_mgr.save(
    fig, caption="Inter-story drift ratio history for all five stories under synthetic GM."
)
plt.show()

# Print peak IDR per story
print("\nPeak IDR per story:")
for story, idr in idr_max_values.items():
    flag = " ⚠️ >1%" if idr > 1.0 else " ✅"
    print(f"  Story {story}: {idr:.4f}%{flag}")

critical = max(idr_max_values, key=idr_max_values.get)
print(f"\n→ IDR_max = {idr_max_values[critical]:.4f}% at Story {critical}")

## 4. Nonlinear Behavior: Global Hysteresis Curve

The **Base Shear vs. Roof Displacement** hysteresis loop is the definitive proof  
that the fiber-section model captures nonlinear cyclic behavior:

- **Closed loops** → energy dissipation (Concrete02 cracking + Steel02 yielding)
- **Stiffness degradation** → the slope decreases in successive cycles
- **Pinching** → characteristic of RC frames with cracked concrete

If this loop were linear (a straight line), the model would be useless for PINN training.

In [ ]:
# ── Global hysteresis: base shear vs. roof displacement ────────────────
roof_disp = df["disp_5"].values
base_shear = df["base_shear"].values

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(roof_disp * 1000, base_shear, color="#2c3e50", lw=0.6, alpha=0.85)

# Mark peak point
idx_peak = int(np.argmax(np.abs(base_shear)))
ax.plot(
    roof_disp[idx_peak] * 1000,
    base_shear[idx_peak],
    "ro",
    ms=8,
    zorder=5,
    label=f"Peak: {base_shear[idx_peak]:.0f} kN @ {roof_disp[idx_peak] * 1000:.1f} mm",
)

ax.set_title("Global Hysteresis: Base Shear vs. Roof Displacement")
ax.set_xlabel("Roof Displacement (mm)")
ax.set_ylabel("Base Shear (kN)")
ax.axhline(0, color="gray", lw=0.5, ls="-")
ax.axvline(0, color="gray", lw=0.5, ls="-")
ax.legend(loc="lower right", fontsize=9)

fig_mgr.save(
    fig, caption="Global hysteresis loop: base shear vs. roof displacement under synthetic GM."
)
plt.show()

print(f"Peak base shear: {np.max(np.abs(base_shear)):.1f} kN")
print(f"Peak roof displacement: {np.max(np.abs(roof_disp)) * 1000:.1f} mm")
print(f"Residual roof displacement: {roof_disp[-1] * 1000:.2f} mm")

## 5. Validation Metrics Summary

Key engineering metrics compiled from the NLTHA output.  
The **dissipated energy** (area enclosed by the hysteresis loop) quantifies  
the cumulative inelastic work — a direct indicator of structural damage.

In [ ]:
# ── Validation metrics ─────────────────────────────────────────────────

# Dissipated energy: integral of V·dδ (trapezoidal rule on hysteresis path)
energy_dissipated = float(np.abs(np.trapezoid(base_shear, roof_disp)))  # kN·m = kJ

# Max displacement per story
max_disp_mm = [float(np.max(np.abs(df[f"disp_{i}"]))) * 1000 for i in range(1, 6)]

# Build summary table
summary = pd.DataFrame(
    {
        "Metric": [
            "PGA",
            "Duration",
            "T₁ (fundamental period)",
            "IDR_max (global)",
            "Critical story",
            "Peak roof displacement",
            "Residual roof displacement",
            "Peak base shear",
            "Dissipated energy",
            "Convergence",
            "Wall-clock time",
        ],
        "Value": [
            f"{pga / 9.81:.3f} g ({pga:.2f} m/s²)",
            f"{meta['ground_motion']['duration']:.2f} s",
            "1.001 s",
            f"{meta['results']['max_drift_overall'] * 100:.4f} %",
            f"Story {meta['results']['critical_story']}",
            f"{np.max(np.abs(roof_disp)) * 1000:.1f} mm",
            f"{roof_disp[-1] * 1000:.2f} mm",
            f"{np.max(np.abs(base_shear)):.1f} kN",
            f"{energy_dissipated:.2f} kJ",
            "✅ Converged (0 retries)" if meta["results"]["converged"] else "❌ Failed",
            f"{meta['results']['wall_clock_s']:.1f} s",
        ],
    }
)

# Per-story breakdown
story_table = pd.DataFrame(
    {
        "Story": list(range(1, 6)),
        "Max IDR (%)": [meta["results"]["max_drift_per_story"][i] * 100 for i in range(5)],
        "Max Displacement (mm)": max_disp_mm,
        "Performance Level": [
            "IO" if d < 0.5 else "LS" if d < 1.0 else "LS→CP" if d < 2.0 else "CP"
            for d in [meta["results"]["max_drift_per_story"][i] * 100 for i in range(5)]
        ],
    }
)

print("=" * 60)
print("   MODEL VERIFICATION — SUMMARY")
print("=" * 60)
display(summary.style.hide(axis="index").set_properties(**{"text-align": "left"}))

print("\n📊 Per-Story Breakdown:")
display(
    story_table.style.hide(axis="index")
    .format({"Max IDR (%)": "{:.4f}", "Max Displacement (mm)": "{:.1f}"})
    .set_properties(**{"text-align": "center"})
)

# Verdict
idr_max_global = meta["results"]["max_drift_overall"] * 100
if idr_max_global < 2.0:
    print("\n✅ VERDICT: Model shows controlled nonlinear behavior (IDR < 2%).")
    print("   → Suitable for PINN training data generation.")
else:
    print("\n⚠️  VERDICT: IDR exceeds CP limit — check model or reduce PGA.")

## 6. Figure Export Verification (300 DPI)

Confirm that all figures were saved correctly by `FigureManager`:
- Sequential naming (`Figure_1.png`, `Figure_2.png`, ...)
- Minimum **300 DPI** resolution (HRPUB requirement)
- File sizes suitable for journal submission

In [ ]:
# ── Verify exported figures ─────────────────────────────────────────────
from PIL import Image

figures_dir = PROJECT_ROOT / "figures"
png_files = sorted(figures_dir.glob("Figure_*.png"))

print(f"Figures directory: {figures_dir}")
print(f"Total figures found: {len(png_files)}\n")

rows = []
for fp in png_files:
    img = Image.open(fp)
    dpi_info = img.info.get("dpi", (0, 0))
    dpi_x = round(dpi_info[0]) if dpi_info else 0
    size_kb = fp.stat().st_size / 1024
    w, h = img.size
    rows.append(
        {
            "File": fp.name,
            "Size (KB)": f"{size_kb:.0f}",
            "Resolution": f"{w}×{h}",
            "DPI": f"{dpi_x}",
            "HRPUB OK": "✅" if dpi_x >= 300 else "❌",
        }
    )
    img.close()

fig_table = pd.DataFrame(rows)
display(fig_table.style.hide(axis="index").set_properties(**{"text-align": "center"}))

# Export caption list for manuscript
captions_md = fig_mgr.export_captions_markdown()
print("\n📄 Caption registry exported to figures/figure_captions.md")
print("\n--- Captions for Section 4 (Results) ---")
print(captions_md)